# 自己進化エージェント：自律エージェントの再訓練のためのクックブック

## 概要

エージェントシステムは、エッジケースの診断と失敗の修正を人間に依存するため、概念実証の後にしばしば停滞に陥ります。このクックブックでは、これらの問題を捕捉し、フィードバックから学習し、改善を本番環境のようなワークフローに反映させる、反復可能な再訓練ループを紹介します。このアプローチは規制された医療文書作成タスクを基盤としていますが、パターンは正確性、監査可能性、迅速な反復が求められる任意のドメインに一般化できます。

### 学習内容
- 自律エージェントが本番環境への準備不足に陥る理由を診断し、測定可能なフィードバック信号でそれを計測する方法
- 迅速な手動反復から完全自動化ループまで、3つのプロンプト最適化戦略を比較し、それぞれをいつ使用するかを理解する
- 人間によるレビュー、LLM-as-judge評価、反復的なプロンプト改良を組み合わせた自己修復ワークフローの構築

### 対象読者
- おもちゃのデモを超えて進む必要があるML/AIエンジニアとソリューションアーキテクト
- 内部ツールや本番パイプラインに適応できる実行可能な成果物を求めるプロダクトチームと配信チーム

### このノートブックの進め方
1. セクション1から始めて、医療ユースケース、ベースラインエージェント、システムアーキテクチャを理解する
2. セクション2を使用してOpenAI Evalsインターフェース内でプロンプト最適化を練習し、構造化されたフィードバックを収集する
3. セクション3を実行して、評価者、評価、再訓練ロジックで最適化ループを自動化する
4. ワークフローを自分の環境に合わせて調整する際は、付録の再利用可能なプロンプト、設定、評価テンプレートを参照する

このノートブックはモジュール式です。再訓練ループを自分のエージェントに適応させる際は、セクションを独立して実行するか、順次実行するかを自由に選択してください。

## 1. ユースケース概要：ヘルスケアにおける自己進化エージェント

### 問題の定義

このクックブックでは、**実世界のユースケース**に焦点を当てます：製薬会社向けの規制文書の作成です。これらの組織は、新薬の承認を得るために規制当局（例：米国食品医薬品局）に広範囲にわたる文書を準備・提出する必要があります。これらの提出物の正確性と迅速性は重要であり、救命治療が患者に届く速度に直接影響するためです。

規制文書の作成は、深い科学的、医学的、コンプライアンス専門知識を必要とする、非常に複雑で反復的、かつ精密性が求められるプロセスです。高度な作成ツールが利用可能であるにもかかわらず、労働集約的で人的エラーが発生しやすいままです。**エージェントシステムは大きな効果を提供**し、研究の統合、コンテンツ生成、文書構造化を支援しますが、事実の正確性と規制遵守を確保するためには人間の専門家が依然として必要です。

主要な課題は、これらのエージェントシステムが反復的に学習し、時間の経過とともにモデルの動作を改善できるフィードバックループを設計することです。このようなシステムは、人間の努力を詳細な修正から高レベルの監督へと段階的にシフトさせ、規制提出に必要な厳格な基準を維持しながら効率を向上させることができます。

### 自己進化エージェント

以下の図は、フィードバック、メタプロンプティング、評価を通じてAIエージェントを継続的に改善する反復プロセスを示しています。このループは、人間の判断またはLLM-as-a-judgeを使用した自動フィードバックを組み合わせて、パフォーマンスを反復的に向上させます。

<img src="../../../images/baseline_agent.png" alt="Self-evolving loop" style="max-width:50%"/>
<br><em>図1 - 自動エージェント改善のための自己進化ループを示す図</em>

プロセスは以下のステップで構成されます：

1. **ベースラインエージェント**  
   プロセスはベースラインエージェントから始まります。このノートブックでは、反復改善ループを説明するために意図的にシンプルな例（文書のセクションを要約するエージェント）を使用します。実世界や企業環境では、ベースラインエージェントははるかに複雑になる可能性があります。生成される要約は、後続の評価と改善のための初期ベンチマークとして機能します。

2. **人間のフィードバック（またはLLM-as-judge）**  
   ベースラインエージェントの出力は、人間のレビュアー（例：本番環境）および/または自動**LLM-as-judge**システムによって評価されます。このステップでは、エージェントが目標をどの程度達成しているかを示す定量的および定性的フィードバックを収集します。例えば、要約の長さをテストしている場合、フィードバックは「要約が長すぎる」または要約が500語以下かどうかを評価する際にevalによって生成される数値スコア（一般的に`0`から`1`の間）になる可能性があります。

3. **Evalsと統合スコア**  
   収集されたフィードバックに基づいて、新しいプロンプトが生成され、評価（**Evals**）を通じてテストされます。これらのテストは事前定義された基準に対するパフォーマンスを測定し、結果は全体的なパフォーマンスを反映する統合スコアに結合されます。スコアが目標閾値（例：`0.8`）を超えるか、最大再試行回数（例：`max_retry = 10`）に達するまでループが続きます。再試行制限に達した場合、手動改善が必要であることがエンジニアに通知されます。

4. **更新されたベースラインエージェント**  
   改善されたバージョンが目標パフォーマンスを達成すると、元のベースラインエージェントを置き換えます。この更新されたエージェントは次の反復の基盤となり、学習、フィードバック、最適化の継続的なサイクルをサポートします。

### データセット概要

評価に使用されるデータセットは、_Sample CMC Section for Hyperpolarized Pyruvate (13C) Injection_から抽出された約70のセクションで構成され、[こちら](https://dctd.cancer.gov/drug-discovery-development/reagents-materials/imaging-ind-resources/documentation/13c-pyruvate-cmc.pdf)で公開されています。このデータセットは、科学的要約と規制遵守の動作の両方をテストするのに適した、現実的でドメイン固有のコンテンツを提供します。

### ベースラインエージェント概要

このクックブックを自己完結型で簡単に再現可能にするため、本質的な複雑さを保持しながら規制作成ユースケースを簡素化しました。本番環境では、典型的な規制作成エージェントは、作成、データ分析、コンプライアンスチェック、引用生成、事実検証などのタスクを担当する複数の専門サブエージェントで構成されます。

このガイドでは、システムの自己修復側面に焦点を当てるため、規制作成エージェントの範囲を狭めます。私たちの規制作成エージェントは2つのサブエージェントで構成されます：
- **要約器**：科学的で簡潔な要約を作成
- **コンプライアンスチェッカー**：各要約を主要な規制要件（例：FDA 21 CFR Part 11）に対して評価

<img src="../../../images/simplified_reg_agent.png" alt="Baseline Agent" style="max-width:50%"/>
<br><em>図2 - AgentBuilder UIで作成されたベースラインエージェント</em>

このクックブックの残りの部分では、要約器エージェントの簡素化されたバージョンを実装しました（以下の**エージェントセットアップ**セクションを参照）。または、AgentBuilderで作成されたエージェントのコードを再利用することもできます。AgentBuilder UIから直接エージェントを再現したい場合は、使用された主要なプロンプトとパラメータは以下の通りです：

- **要約器エージェント：** このエージェントはファイル検索ツールを使用し、[CMC PDF]("data/c13_pyruvate_sample_CMC_from_UCSF.pdf")がベクトルストアにアップロードされました。
> _プロンプト:_ "ベクトルストアにアップロードされた{{state.cmc_pdf}}から{{workflow.input_as_text}}セクションを要約してください。"

- **コンプライアンスチェッカーエージェント：**
> _プロンプト:_ "以下の要約がFDA 21 CFR Part 11に準拠していることを確認してください：{{input.output_text}}。要約が準拠している場合は_Compliant_を返してください。そうでなければ_This section needs to be manually summarized_を返してください。"

両方のエージェントはデフォルトパラメータで設定されました - GPT-5を使用し、低い推論努力、テキスト出力形式です。

### 評価アプローチ

ベースラインエージェントを評価するには、主に2つのアプローチがあります：

1. **人間のフィードバックの収集。** このアプローチは、OpenAI Evalsプラットフォーム（または特定のアプリケーション用に構築されたカスタムUI）を通じて人間のユーザーからフィードバックを収集することを含みます。本番設定や、専門家（SME）が実世界のシナリオでツールと対話するツールのパイロット時に最適です。この方法は、開発中に特定されなかった可能性のあるエッジケースを発見するのに役立ちます。Evalsプラットフォームでは、ユーザーは親指を上げる/下げる評価を提供し、要約に関する定性的フィードバックを共有できます。

2. **LLM-as-a-Judgeの使用。** このオプションは通常、開発段階で使用され、SMEの時間を必要とせずに高速なフィードバックループを可能にします。**LLM-as-a-judge**は、LLMを使用してエージェントの出力を事前定義された基準に基づいて自動的に評価・スコア化します。また、モデルドリフトの監視（例：本番環境）やモデルとモデルバージョン間の変更の検証（例：`gpt-5`から`gpt-5-mini`への切り替え）にも使用できます。

このクックブックでは両方のアプローチを実演します：
- **セクション2**では、手動プロンプト最適化のためのプラットフォームUIアプローチを示します
- **セクション3**では、LLM-as-a-judgeを使用した完全自動化APIアプローチを実装します

_注：Evalsプラットフォームは、ユーザーフィードバックをプログラム的に取得するAPIをまだ提供していません。_

## 2. OpenAI Evalsプラットフォームの使用

OpenAI Evalsプラットフォームは、プロンプト最適化と評価のための直感的なインターフェースを提供します。このセクションでは、データセットのアップロードから反復的なプロンプト改善まで、完全なワークフローを実演し、自動化ソリューションを実装する前にプラットフォームの視覚的インターフェースを活用してプロンプトを最適化する方法を示します。

### ステップ1: データセットのアップロード

OpenAI Evaluationプラットフォームの使用を開始するには、まずデータセットをアップロードする必要があります：

1. **+ Create**ボタンをクリック
2. データセット名を定義
3. CSVファイルをアップロードし、保持する列を選択
4. アップロード

データセットには、要約が必要な文書または文書セクションを含める必要があります。各行は、システムによって処理される1つの入力を表します。

### ステップ2: データの探索

アップロード後、データセットを探索できます。データセット名をクリックして、アップロードされたデータを探索します。これにより、プロンプト設定に進む前に、データが適切にフォーマットされ、期待されるコンテンツが含まれていることを確認できます。

### ステップ3: 初期プロンプトの設定

ここで初期システムプロンプトを定義し、データがモデルを通してどのように流れるかを設定します。

<img src="../../../images/prompt_input.png" alt="Platform Prompt Configuration" style="max-width:50%">
<br><em>図3 - モデル設定、変数、システムメッセージ設定を示すプラットフォームの「新しいプロンプト」インターフェース</em>

#### 設定手順

1. **システムプロンプト**: モデルのタスクと動作を定義するシステムメッセージを追加（このプロンプトが最適化されます）
2. **ユーザープロンプトテンプレート**: ユーザーメッセージのプロンプトメッセージテンプレートを追加し、データセットの実際のデータに置き換えられる`{{<column_name>}}`などの変数を使用
3. **モデル選択**: 生成用のモデルを選択（例：gpt-4.1、gpt-5）
4. **Temperature**: 創造性と決定論のバランスを設定

最適化プロセスの威力を実証するために、非常にシンプルなプロンプトから始めることができます。例えば、単に「summarize」から始めることで、システムが最小限の出発点からどのように進化できるかを示します。

### ステップ4: 出力の生成

プロンプトが設定されたら、データセット全体で出力を生成する準備が整いました。プロンプトは行ごとに1回実行され、新しい**output**列に出力が生成されます。

1. **「Generate Output」**をクリック
2. プラットフォームがすべてのサンプルに対してプロンプトを実行
3. 結果が新しい**Output**列に表示

プラットフォームはデータセットの各行を処理し、テンプレート変数を実際の値に置き換え、システムプロンプトでモデルを呼び出します。これにより、評価可能な出力のベースラインが作成されます。

### ステップ5: レビューと評価

評価は、プロンプト改善を導くための構造化されたフィードバックを提供する場所です。

#### 出力のレビュー

1. **評価列の追加**（自動的に追加されていない場合） - 「Columns」→「Annotations」→「Add」をクリック：
   - **Rating** - バイナリ（良い/悪い）または数値評価
   - **Feedback** - 改善が必要な点を説明するテキスト

2. **評価とフィードバックの提供** - 各出力に対する評価を追加

   出力の品質に応じて、良いまたは悪い評価を選択し、回答をどのように改善したいかに基づいてスコアを説明できます。例えば：

      > (評価) | フィードバック
      > - (良い) 良いが、回答のみを提供すべき。出力にはヘッダーや回答以外のテキストを含めるべきではない。
      > - (悪い) 情報は良いが、箇条書きで提示すべき。
      > - (良い) 良い要約；明確である。
      > - (悪い) 読みやすさを向上させるため、回答時に箇条書きを使用する。各サブセクションを個別に要約する。

3. **注釈の保存** - フィードバックは評価実行と共に保存

<img src="../../../images/feedback.png" alt="Platform Evaluation Interface" style="max-width:50%">
<br><em>図4 - 注釈用の評価とフィードバック列を持つ生成された出力を示す評価インターフェース</em>

この構造化されたフィードバックが、自動プロンプト最適化の基盤となります。

### ステップ6: プロンプトの最適化

フィードバックを収集した後、プラットフォームは自動的に改善されたプロンプトを生成できます。

1. **「Optimize」**をクリック
2. 新しいタブで新しいプロンプトバージョンが生成
3. **「View Prompt」**をクリックして改善されたバージョンを確認

<img src="../../../images/updated_prompt.png" alt="Platform Optimized Prompt" style="max-width:50%">
<br><em>図5 - プラットフォームによって生成された改善されたプロンプト、詳細な指示と要件を示している</em>

### ステップ7: 反復と比較

改善されたプロンプトの準備ができたら、新しい反復を開始して改善を測定します。

1. **「Generate Output」**をクリック
2. 新しい結果をレビューし、残りの問題についてフィードバックを提供
3. 必要に応じて再度**「Optimize」**をクリック
4. 満足するまで繰り返し

プラットフォームのタブ構造により、反復間でのパフォーマンスを比較できます。初期プロンプトから最適化されたバージョンまで、出力がどのように進化したかを簡単に確認できます。

<img src="../../../images/updated_prompt_feedback.png" alt="Platform Updated Prompt Feedback" style="max-width:50%">
<br><em>図6 - 最適化されたプロンプトのフィードバックと評価結果、出力品質の改善を示している</em>

#### 反復を停止するタイミング

以下の条件まで最適化サイクルを続けます：
- **品質閾値に到達**: 出力の80%以上が肯定的なフィードバックを受ける
- **収穫逓減**: 新しい反復で最小限の改善しか見られない
- **特定の問題が解決**: 特定されたすべての失敗モードが対処される

このプラットフォームベースのアプローチは、自動化実装に移る前にプロンプト最適化を理解するための優れた基盤を提供します。視覚的インターフェースにより、変更の影響を簡単に確認し、最適化プロセスを理解できます。

## 3. LLM-as-a-Judgeを使用した自己進化ループ

このセクションでは、OpenAI APIを通じてLLM-as-a-Judgeを使用した完全自動化評価ワークフローを紹介し、ユーザーインターフェースの必要性を排除します。このアプローチにより、エージェントのパフォーマンスのスケーラブルでプログラマティックな評価が可能になり、本番環境での迅速な反復と継続的なモデル監視をサポートします。

In [ ]:
# gepa and litellm are only required for the Section 4.b (prompt optimization with GEPA)
%pip install --upgrade openai openai-agents pydantic pandas gepa litellm python-dotenv -qqq 
%load_ext dotenv
%dotenv

# Place your API key in a file called .env
# OPENAI_API_KEY=sk-...


### 評価の作成

ベースライン要約エージェントを評価するために、決定論的チェックと意味的判断のバランスを取る4つの補完的な評価者を使用します。

| 評価者 | タイプ | 合格閾値 | チェック内容 | 理由 |
|---|---|---:|---|---|
| 化学物質名 | `python` | 0.8 | セクション内の正確な化学物質名が要約に含まれているか | 重要なドメインエンティティの保持を強制し、要約が化学的に意味のある用語を省略しないようにする |
| 要約の長さ | `python` | 0.85 | 期待される100語の長さからの逆偏差 | 要約を簡潔で比較可能に保ち、内容の質の低さを隠す可能性のある冗長性を削減する |
| コサイン類似度 | `text_similarity` | 0.85 | セクションと要約テキスト間のコサイン類似度 | 要約が意味的に逸脱するのではなく、ソースコンテンツに固定されることを保証する |
| LLM審査員 | `score_model` | 0.85 | 評価者として機能するモデルからのルーブリック駆動スコア | ルールベースのメトリクスが見逃す微妙な品質シグナルを捉え、全体的な堅牢性を向上させる |

**注記**
- 2つのPython評価者は、ドメインの忠実性と長さの規律を早期に捉え、意味的調整前に最適化を安定化させます。
- テキスト類似度は、ソースから逸脱する表面的な言い換えを防ぎます。
- LLM審査員は、決定論的チェックをすり抜けるエッジケースに対する包括的な安全装置を提供します。

In [ ]:
import os
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

data_source_config = {
    "type": "custom",
    "item_schema": {
        "type": "object",
        "properties": {"section": {"type": "string"}, "summary": {"type": "string"}},
        "required": ["section", "summary"],
    },
    "include_sample_schema": False,
}

testing_criteria = [
    {
        "type": "python",
        "name": "chemical_name_grader",
        "image_tag": "2025-05-08",
        "pass_threshold": 0.8,
        "source": r"""def grade(sample: dict, item: dict) -> float:
    section = item["section"]
    summary = item["summary"]
    CHEMICALS_MASTER = ["[1-¹³C]Pyruvic acid","[1-¹³C]Pyruvate","¹²C Pyruvic acid","Sodium [1-¹³C]pyruvate","Sodium pyruvate (¹²C)","AH111501 (Trityl radical)","Tris{8-carboxyl-2,2,6,6-tetra[2-(1-methoxyethyl)]-benzo(1,2-d:4,5-d’)bis(1,3)dithiole-4-yl}methyl acid","AH111501 sodium salt","Methyl, tris[8-carboxy-2,2,6,6-tetrakis(2-methoxyethyl)benzo[1,2-d:4,5-d’]bis[1,3]dithiol-4-yl]-, trisodium salt","AH111501 trisodium salt","AH111576","2,2′,2″,2‴-(4,8-Dibromobenzo[1,2-d:4,5-d′]bis([1,3]dithiole)-2,2,6,6-tetrayl)tetraethanol","AH111586","4,8-Dibromo-2,2,6,6-tetrakis(2-methoxyethyl)benzo[1,2-d:4,5-d′]bis([1,3]dithiole)","AH111709","AH111743","AH112615","4,4-Bis-hydroxymethyl-2-methyl-oxazolidine-2-carboxylic acid","AH112623","Parapyruvate","2-Hydroxy-2-methyl-4-oxo-pentanedioic acid","AH113127","(4-Hydroxymethyl-oxazolidin-4-yl)-methanol","AH113462/E","Enol lactone","AH113462/K","Keto lactone","Acetyl bromide","Methanol","Dimethyl sulfoxide","DMSO","Tetrahydrofuran","THF","Acetonitrile","ACN","Diethyl ether","Et₂O","N,N-Dimethylacetamide","DMA","1,3-Dimethyl-2-imidazolidinone","DMI","Hydrochloric acid","HCl","Sodium hydroxide","NaOH","Disodium ethylenediaminetetraacetate","Na₂EDTA","Ethylenediaminetetraacetic acid","EDTA","Tris(hydroxymethyl)aminomethane","TRIS","Trometamol","Trifluoroacetic acid","TFA","Toluene","Heptane","Ethyl acetate","Ethanol","Water","H₂O","Sodium chloride","NaCl","Cuprous [1-¹³C]cyanide","Cu¹³CN","Gadolinium","Gd","Tin","Sn","Phosphorus","P","Carbon dioxide","CO₂","Sodium [1-13C]pyruvate","[1-13C]Pyruvic acid","1-13C pyruvate"]

    # Identify the chemicals present in the section
    present = [chem for chem in CHEMICALS_MASTER if chem in section]

    # If no chemicals present, consider it satisfied
    if not present:
        return 1.0

    correct = 0
    for chem in present:
        # Only count as correct if the exact chemical string appears in the summary
        if chem in summary:
            correct += 1

    return correct / len(present)""",
    },
    {
    "type": "python",
    "name": "word_length_deviation_grader",
    "image_tag": "2025-05-08",
    "pass_threshold": 0.85,
    "source": r"""
def grade(sample: dict, item: dict) -> float:
    summary = item["summary"]
    word_count = len(summary.split())
    
    expected_summary_length = 100
    tolerance = 0.2  # 20% band around target
    
    # relative deviation
    deviation = abs(word_count - expected_summary_length) / expected_summary_length
    
    # If within tolerance band → full score
    if deviation <= tolerance:
        return 1.0
    
    # Outside band → score decays linearly, capped at 0
    # e.g., deviation 0.3 → score 0.8, deviation 1.0+ → 0.0
    score = 1.0 - (deviation - tolerance)
    return max(0.0, score)
""",
},
    {
        "name": "cosine_similarity",
        "type": "text_similarity",
        "input": "{{ item.summary }}",
        "reference": "{{ item.section }}",
        "evaluation_metric": "cosine",
        "pass_threshold": 0.85,
    },
    {
        "name": "llm_as_judge",
        "type": "score_model",
        "model": "gpt-4.1",
        "input": [
            {
                "role": "system",
                "content": (
                    "You are an expert technical summarization evaluator. "
                    "Evaluate whether the summary captures and preserves the important technical facts and specific details from the section, allowing for occasional minor rewording or omissions of less important points, but not major technical inaccuracies or information loss.\n\n"
                    "Scoring Guidelines:\n"
                    "- Return a numerical score between 0 and 1 (with up to two decimal places).\n"
                    "- A score of 1 means the summary is almost flawless: it is comprehensive, highly faithful, and technically accurate, with virtually no important or meaningful details missing, and no significant misstatements or distortions.\n"
                    "- 0.75-0.99 indicates excellent work: all main facts are represented, but there may be trivial omissions or very minor rewording that do not materially affect understanding.\n"
                    "- 0.5-0.75 indicates good but imperfect: most technical information is retained and correctly presented, some less critical details might be missing or slightly rephrased, but overall fidelity is preserved.\n"
                    "- 0.3-0.5 means significant information is missing, or some technical inaccuracies are present, but the summary retains a reasonable portion of key facts.\n"
                    "- 0.0-0.3 means there are major omissions, misunderstandings, or a failure to capture the most important technical content.\n\n"
                    "Respond only with a single number between 0 and 1 indicating summary quality by these criteria."
                ),
            },
            {
                "role": "user",
                "content": (
                    "Section:\n{{item.section}}\n"
                    "Summary:\n{{sample.output_text}}"
                ),
            },
        ],
        "range": [0, 1],
        "pass_threshold": 0.85,
    },
]

eval = client.evals.create(
    name="self_evolving_eval",
    data_source_config=data_source_config,
    testing_criteria=testing_criteria,
)
print(f"Created Eval: {eval.id}")

出力にeval ID（例：`eval_...`）が表示されるはずです。これは、先ほど作成したevalのIDです（以下に示すとおり）

<img src="../../../images/eval_set_config.png" alt="Platform Eval Configuration" style="max-width:50%">
<br><em>図7 - データソース設定とテスト基準設定を表示するプラットフォームのEvalインターフェース</em>

### グレーダーのスコアリングと解析

次に、要約エージェントの出力に対してevalを実行し、evalのグレーダースコアの結果を解析する必要があります。これを行うために、いくつかのヘルパー関数を使用します：
- `run_eval`: 適切なフォーマットでevals APIを呼び出すシンプルなランナー
- `poll_eval_run`: スケジュールされたeval実行の完了を待機するポーリングユーティリティ
- `parse_eval_run_output`: eval実行を解析し、フィードバックループ用の構造化された出力を返す

In [ ]:
import time
import json

def run_eval(eval_id: str, section: str, summary: str):
  """Creates a run of the eval with the input section and output summary."""
  return client.evals.runs.create(
    eval_id=eval_id,
    name="self-evolving-eval",
    data_source={
      "type": "jsonl",
      "source": {
        "type": "file_content",
        "content": [
          {
            "item": {
              "section": section,
              "summary": summary,
            }
          }
        ],
      },
    },
  )


def poll_eval_run(eval_id: str, run_id: str, max_polls = 10):
    """
    Polls the evaluation run until completion or timeout.

    This function exists to handle asynchronous behavior in the eval service by
    periodically checking run status. It balances responsiveness and resource use by
    polling at fixed intervals rather than blocking indefinitely. The retry limit
    prevents runaway loops in cases where the service never returns a completed status.
    """
    run = None
    for attempt in range(1, max_polls + 1):
        run = client.evals.runs.retrieve(eval_id=eval_id, run_id=run_id)
        if run.status == "completed":
            break
        if attempt == max_polls:
            print("Exceeded retries, aborting")
            break

        time.sleep(5)

    run_output_items = client.evals.runs.output_items.list(
        eval_id=eval_id, run_id=run_id
    )
    return run_output_items


def parse_eval_run_output(items):
    """Extract all grader scores and any available conclusion outputs."""
    all_results = []

    for item in items.data:
        for result in item.results:
            grader_name_full = result.name
            score = result.score
            passed = result.passed
            reasoning = None
            try:
                sample = result.sample
                if sample:
                    content = result.sample["output"][0]["content"]
                    content_json = json.loads(content)
                    steps = content_json["steps"]
                    reasoning = " ".join([step["conclusion"] for step in steps])
            except Exception:
                pass

            all_results.append(
                {
                    "grader_name": grader_name_full,
                    "score": score,
                    "passed": passed,
                    "reasoning": reasoning,
                }
            )

    return all_results

これで、先ほど作成したeval IDを使用して、任意の入力セクションと要約出力に対してgraderを実行できます。これは、プロンプト最適化ルーチンを開始するフィードバックループの基盤を形成します。

### Eval実行の実行

セクションと生成された要約を直接提供して、evalをテストしてみましょう。

In [ ]:
EVAL_ID = eval.id #Created eval ID from above cell
SECTION = "3.2.S.1 General Information ([1-13C]pyruvic acid) The active ingredient in Hyperpolarized Pyruvate (13C) Injection is hyperpolarized [1-13C]pyruvate. The drug substance is defined as [13C]pyruvic acid, which is neutralized to [1-13C]pyruvate during the compounding process. In several pre-clinical and clinical studies and during evaluation of stability, pyruvic acid has been used instead of [1-13C]pyruvic acid (see Sections 3.2.P.2.2.1 Formulation Development for Hyperpolarized Pyruvate (13C) Injection and Section 8.1 Introduction for Item 8 Pharmacology and Toxicology Info). In the Section 3.2.S Drug Substance, data are presented for both pyruvic acid and for [1-13C]pyruvic acid. For simplicity, the terminology used in headings and captions is [1-13C]pyruvic acid. Batches containing pyruvic acid are specified by footnotes. 3.2.S.1.1 Nomenclature ([1-13C]pyruvic acid) The drug substance used for compounding of Hyperpolarized Pyruvate (13C) Injection is [1-13C]pyruvic acid. Company code: W6578 Chemical name: [1-13C]pyruvic acid CAS registry number: 127-17-3 3.2.S.1.2 Structure ([1-13C]pyruvic acid) Figure 1 Structure of [1-13C]pyruvic acid Molecular formula: C H O 3 4 3 Molecular weight: 89.06 3.2.S.1.3 General Properties ([1-13C]pyruvic acid) Appearance: Colorless to yellow, clear, viscous liquid pKa:Ka:aranWater solubility: Complete The structure of [1-13C]pyruvic acid has been confirmed by spectroscopic analysis (see Section 3.2.S.3.1 Elucidation of Structure and other Characteristics)."
SUMMARY = "The active ingredient in Hyperpolarized Pyruvate (13C) Injection is hyperpolarized [1-13C]pyruvate, derived from [1-13C]pyruvic acid (neutralized during compounding). Both pyruvic acid and [1-13C]pyruvic acid were used in studies and stability evaluations, but the documentation refers to [1-13C]pyruvic acid unless otherwise noted. The drug substance ([1-13C]pyruvic acid, CAS 127-17-3) is a colorless to yellow, clear, viscous liquid with a molecular formula C3H4O3 and molecular weight 89.06. Its structure has been confirmed by spectroscopic analysis, and it is completely soluble in water."

eval_run = run_eval(EVAL_ID, section=SECTION, summary=SUMMARY)
run_output = poll_eval_run(eval_id=EVAL_ID, run_id=eval_run.id)

grader_scores = parse_eval_run_output(run_output)
print(grader_scores)

出力には以下のようなグレーダースコアのリストが表示されるはずです：

```[{'grader_name': 'chemical_name_grader-<uuid>', 'score': 0.5, 'passed': False, 'reasoning': None}, {'grader_name': 'word_length_deviation_grader-<uuid>', 'score': 0.8, 'passed': True, 'reasoning': None}, {'grader_name': 'cosine_similarity-<uuid>', 'score': 0.9104484223477793, 'passed': True, 'reasoning': None}, {'grader_name': 'llm_as_judge-<uuid>', 'score': 0.8, 'passed': True, 'reasoning': 'The summary needs to include specific details from the section. Part of the essential information is captured. Key pieces of information are missing. Not all relevant structural information is included.'}]```

このスクリプトを実行すると、`chemical_name_grader`を除いて、ほとんどのグレーダーが合格していることがわかります。次に、要約エージェントを改善する機会をプログラム的に認識します。

_注意: ローカルで実行する場合、最初は`chemical_name_grader`以外のグレーダーも失敗する可能性があります。これは正常な動作です。グレーダーは最初は失敗することがありますが、フィードバックループを通じて結果が改善されるはずです。初期の失敗は、モデルがより正確な結果に収束する前に応答を調整していることを単純に反映しています。_


### ダッシュボードの可観測性
評価の実行と結果はOpenAIダッシュボードでも確認できます：

<img src="../../../images/eval_dashboard.png" alt="Eval dashboard" style="max-width:50%">
<br><em>図8 - 評価実行と結果を表示する評価ダッシュボード。</em>


特定の評価実行の詳細を掘り下げることもできます：
<img src="../../../images/eval_run_results.png" alt="Eval results" style="max-width:50%">
<br><em>図9 - グレーダースコアとパフォーマンス指標を表示する詳細な評価実行結果。</em>


## エージェントのセットアップ

評価とグレーダーの設定が完了したので、要約エージェントに戻ることができます。
簡単にするため、以下にシンプルなエージェントのコードを提供します。図2に示されているように`AgentBuilder`を使用して、UIからコードをエクスポートすることもできます。


また、プロンプトを最適化するためのメタプロンプト最適化エージェントと、プロンプトバージョンを処理するためのシンプルなユーティリティも必要になります：
- `PromptVersionEntry`: プロンプトとメタデータを本番環境で変更される際に追跡するために使用されるpydanticモデル
- `VersionedPrompt`: プロンプトバージョンを追跡するユーティリティクラス。これは、プロンプトの進化を分析し、回帰が発生した場合のフォールバック履歴を確保するために、本番環境で重要になります

In [ ]:
from datetime import datetime
from typing import Any, Optional

from pydantic import BaseModel, Field, ConfigDict, field_validator

class PromptVersionEntry(BaseModel):
    """Data model for a prompt and associated data for observability"""
    version: int = Field(
        ..., ge=0, description="Version number of the prompt (increments)"
    )
    model: str = Field(
        "gpt-5",
        min_length=1,
        description="The model version to use for this version of the prompt, defaults to gpt-5",
    )
    prompt: str = Field(
        ..., min_length=1, description="The prompt text for this version"
    )
    timestamp: datetime = Field(
        default_factory=datetime.utcnow,
        description="UTC timestamp when this version was created",
    )
    eval_id: Optional[str] = Field(
        None, description="ID of the evaluation associated with this prompt version"
    )
    run_id: Optional[str] = Field(
        None, description="ID of the run associated with this prompt version"
    )
    metadata: Optional[dict[str, Any]] = Field(
        None, description="Free-form metadata dict (e.g., section, summary)"
    )

    model_config = ConfigDict(
        str_strip_whitespace=True, validate_assignment=True, extra="forbid"
    )

    @field_validator("prompt")
    @classmethod
    def prompt_not_blank(cls, v: str) -> str:
        if not v.strip():
            raise ValueError("prompt must not be blank or only whitespace")
        return v


class VersionedPrompt:
    """Manages a collection of prompt versions and provides controlled updates and rollbacks."""
    def __init__(
        self,
        initial_prompt: str,
        model: Optional[str] = "gpt-5",
        eval_id: Optional[str] = None,
        run_id: Optional[str] = None,
        metadata: Optional[dict[str, Any]] = None,
    ):
        if not initial_prompt or not initial_prompt.strip():
            raise ValueError("initial_prompt must be non-empty")
        self._versions: list[PromptVersionEntry] = []
        first_entry = PromptVersionEntry(
            version=0,
            prompt=initial_prompt,
            model=model,
            eval_id=eval_id,
            run_id=run_id,
            metadata=metadata,
        )
        self._versions.append(first_entry)

    def update(
        self,
        new_prompt: str,
        model: Optional[str] = "gpt-5",
        eval_id: Optional[str] = None,
        run_id: Optional[str] = None,
        metadata: Optional[dict[str, Any]] = None,
    ) -> PromptVersionEntry:
        if not new_prompt or not new_prompt.strip():
            raise ValueError("new_prompt must be non-empty")

        version = self.current().version + 1
        entry = PromptVersionEntry(
            version=version,
            prompt=new_prompt,
            model=model,
            eval_id=eval_id,
            run_id=run_id,
            metadata=metadata,
        )
        self._versions.append(entry)
        return entry

    def current(self) -> PromptVersionEntry:
        return self._versions[-1]

    def revert_to_version(self, version: int) -> PromptVersionEntry:
        idx = None
        for i, entry in enumerate(self._versions):
            if entry.version == version:
                idx = i
                break

        if idx is None:
            raise ValueError(f"No version found with version={version}")

        self._versions = self._versions[: idx + 1]
        return self._versions[-1]

次に、要約とプロンプト最適化の開始エージェントを作成します。

_注意：要約エージェントは本番環境で進化することが予想されるため、プロンプトの変更を追跡するラッパーを作成しました。メタプロンプトエージェントのプロンプトは、このクックブックの目的においては静的なままとなります。_

In [ ]:

from agents import Agent

METAPROMPT_TEMPLATE = """
# Context:
## Original prompt:
{original_prompt}

## Section:
{section}

## Summary:
{summary}

## Reason to improve the prompt:
{reasoning}

# Task:
Write a new summarization prompt that is significantly improved and more specific than the original.  
The new prompt should instruct the model to produce concise yet comprehensive technical summaries that precisely preserve all explicit information from the source text. It should emphasize the inclusion of all named entities, quantities, compounds, and technical terminology without paraphrasing or omission. The resulting prompt should read like a clear, directive system message for a technical summarization assistant—structured, unambiguous, and generalizable across scientific or regulatory document sections.
"""

metaprompt_agent = Agent(
    name="MetapromptAgent", instructions="You are a prompt optimizer."
)

summarization_prompt = VersionedPrompt(
    initial_prompt="""You are a summarization assistant.
Given a section of text, produce a summary."""
)

def make_summarization_agent(prompt_entry: PromptVersionEntry) -> Agent:
    return Agent(
        name="SummarizationAgent",
        instructions=prompt_entry.prompt,
        model=prompt_entry.model,
    )

summarization_agent = make_summarization_agent(summarization_prompt.current())

# Cache eval results by section + summary so repeated attempts do not trigger redundant grader runs.
eval_cache: dict[tuple[str, str], list[dict[str, Any]]] = {}

# Track the highest-scoring candidate that also passes the lenient score threshold.
best_candidate: dict[str, Any] = {
    "score": float("-inf"),
    "prompt": summarization_prompt.current().prompt,
    "model": summarization_prompt.current().model,
    "summary": None,
    "metadata": None,
    "version": summarization_prompt.current().version,
    "passed_lenient": False,
    "total_score": float("-inf"),
}

# Aggregate per-version performance so we can pick the strongest total scorer at the end.
aggregate_prompt_stats: dict[int, dict[str, Any]] = {}



### オーケストレーションとモニタリング

これまでに以下のものを作成しました：
- 出力を評価し、各評価者のスコアを生成する4つの評価者を持つEvals
- プロンプトとモデルの変更を追跡するためのバージョン管理されたプロンプトクラスを持つ要約エージェント
- 推論のセットに基づいてプロンプトの更新を試みるメタプロンプト最適化エージェント

これらの異なる機能を組み合わせて、OpenAIダッシュボードでのAgentトレーシングを使用した自己進化ループをオーケストレートできます。

これは簡略化された例であることに注意してください。実際のシナリオでは、最適化の試行に対するガードレールを確保し、ガードレールがトリガーされた際に人間に警告する仕組みが必要になります。

_注意：クックブックの実用的な制限により、静的データセットを使用してデータストリームをシミュレートし、真の観測可能性の代わりに`print`文を使用しています。_

### オーケストレーション・ユーティリティ

前のセクションと同様に、フィードバックループのオーケストレーションロジックを管理するためのユーティリティを作成します。

In [ ]:
import asyncio
from typing import Any, Optional
from agents import Runner

LENIENT_PASS_RATIO = 0.75 # 75% of graders must pass (binary) 
LENIENT_AVERAGE_THRESHOLD = 0.85 # 85% average score across graders 

def reset_best_candidate() -> None:
    """Reset the best candidate tracker for a new optimization run."""
    global best_candidate

    current = summarization_prompt.current()
    best_candidate = {
        "score": float("-inf"),
        "prompt": current.prompt,
        "model": current.model,
        "summary": None,
        "metadata": None,
        "version": current.version,
    }

def reset_best_trackers() -> None:
    """Reset both the best-candidate tracker and aggregate stats."""
    reset_best_candidate()
    aggregate_prompt_stats.clear()


def update_best_candidate(
    *,
    average_score: Optional[float] = None,
    prompt_text: str,
    model_name: str,
    summary_text: str = None,
    metadata: dict[str, Any] = None,
    lenient_passed: bool = False,
    prompt_version: int = None,
    total_score: Optional[float] = None,
    score: Optional[float] = None,
) -> None:
    """Persist the best lenient-passing candidate."""
    global best_candidate

    if prompt_version is None:
        prompt_version = summarization_prompt.current().version

    if average_score is None:
        average_score = score

    if average_score is None:
        return

    if lenient_passed:
        best_candidate.update(
            {
                "score": average_score,
                "prompt": prompt_text,
                "model": model_name,
                "summary": summary_text,
                "metadata": metadata,
                "version": prompt_version,
                "total_score": total_score if total_score is not None else average_score,
            }
        )


def apply_best_candidate_if_needed() -> Agent:
    """Ensure summarization_prompt reflects the best prompt candidate."""
    if best_candidate["score"] > float("-inf"):
        current = summarization_prompt.current()
        target = best_candidate
        # Only update if different
        if (
            current.prompt != target["prompt"]
            or current.model != target["model"]
            or current.version != target.get("version")
        ):
            summarization_prompt.update(
                new_prompt=target["prompt"],
                model=target["model"],
                metadata=target.get("metadata"),
            )
            target["version"] = summarization_prompt.current().version
        return make_summarization_agent(summarization_prompt.current())

    return make_summarization_agent(summarization_prompt.current())


def record_aggregate_prompt_score(
    *,
    prompt_version: int,
    prompt_text: str,
    model_name: str,
    average_score: float,
    total_score: Optional[float] = None,
) -> None:
    """Accumulate per-version grader scores for aggregate selection."""
    stats = aggregate_prompt_stats.setdefault(
        prompt_version,
        {
            "version": prompt_version,
            "prompt": prompt_text,
            "model": model_name,
            "total_score": 0.0,
            "total_average": 0.0,
            "count": 0,
        },
    )
    stats["total_score"] += total_score if total_score is not None else average_score
    stats["total_average"] += average_score
    stats["count"] += 1
    stats["prompt"] = prompt_text
    stats["model"] = model_name


def select_best_aggregate_prompt() -> Optional[dict[str, Any]]:
    """Return the prompt version with the highest cumulative score."""
    if not aggregate_prompt_stats:
        return None
    return max(
        aggregate_prompt_stats.values(),
        key=lambda entry: (
            entry.get("total_score", float("-inf")),
            entry.get("version", -1),
        ),
    )


async def get_eval_grader_score(eval_id: str, section: str, summary: str):
    """Retrieve grader scores for a section-summary pair with caching."""
    cache_key = (section, summary)
    if cache_key in eval_cache:
        return eval_cache[cache_key]

    eval_run = run_eval(eval_id=eval_id, section=section, summary=summary)
    run_output = poll_eval_run(eval_id=eval_id, run_id=eval_run.id)
    results = parse_eval_run_output(run_output)
    eval_cache[cache_key] = results
    return results


def calculate_grader_score(grader_scores):
    """Simple average score of all graders from the eval."""
    if not grader_scores:
        return 0.0

    score_sum = 0.0
    for entry in grader_scores:
        score_sum += entry.get("score", 0.0)

    return score_sum / len(grader_scores)



def calculate_total_grader_score(grader_scores):
    """Sum of all grader scores for aggregate tracking."""
    if not grader_scores:
        return 0.0

    return sum(entry.get("score", 0.0) for entry in grader_scores)


DEFAULT_PASSING_FEEDBACK = (
    "All graders passed; tighten factual coverage, chemical completeness, and conciseness."
)


def is_lenient_pass(grader_scores, average_score: float) -> bool:
    if not grader_scores:
        return False

    passed_count = sum(1 for entry in grader_scores if entry.get("passed"))
    total_graders = len(grader_scores)

    if total_graders and (passed_count / total_graders) >= LENIENT_PASS_RATIO:
        return True
    return average_score >= LENIENT_AVERAGE_THRESHOLD


def collect_grader_feedback(grader_scores):
    """Consolidate grader reasoning into actionable feedback for the metaprompt agent."""
    feedback_lines = []

    for entry in grader_scores:
        grader = entry.get("grader_name", "")
        passed = entry.get("passed", False)
        reasoning = entry.get("reasoning")

        if not passed:
            if grader.startswith("chemical_name_grader"):
                feedback_lines.append(
                    "Not all chemical names in the input section were included in the summary."
                )
            elif grader.startswith("word_length_deviation_grader"):
                feedback_lines.append(
                    "The summary length deviates too much from the expected length."
                )
            elif grader.startswith("cosine_similarity"):
                feedback_lines.append(
                    "The summary is not sufficiently similar to the source section (cosine similarity too low)."
                )
            elif grader.startswith("llm_as_judge") and reasoning:
                feedback_lines.append(reasoning)

    if not feedback_lines:
        feedback_lines.append(DEFAULT_PASSING_FEEDBACK)

    return "".join(feedback_lines)



### 自己進化ループ

要約リクエストのストリームをシミュレートするために、準備されたデータセットを入力し、素朴なプロンプトから最適化がどのように進化するかを観察します。

> 参照されているdataset.csvはGithubリポジトリで見つけることができます。

In [ ]:
import pandas as pd

from agents import Agent, trace

EVAL_ID = eval.id #Created eval ID from above cell
MAX_OPTIMIZATION_RETRIES = 3

async def self_evolving_loop(summarization_agent: Agent) -> Agent:
    print(f"Starting self-evolving loop | Initial prompt v{summarization_prompt.current().version}")
    print(f"Prompt:{summarization_prompt.current().prompt}")
    print("-" * 80)

    reset_best_trackers()
    df = pd.read_csv("data/dataset.csv")

    with trace("Self-evolving Optimization Workflow"):
        for _, row in df.head().iterrows():
            content = row.get("content")
            if pd.isna(content) or (isinstance(content, str) and not content.strip()):
                continue

            section_number = str(row["section_number"])
            section = str(content)
            current_version = summarization_prompt.current().version

            print(f"[Section {section_number}] Using prompt v{current_version}")

            optimization_success = False

            for attempt in range(1, MAX_OPTIMIZATION_RETRIES + 1):
                print(f"  Attempt {attempt}: evaluating summary...")

                summary_result = await Runner.run(summarization_agent, section)
                summary = summary_result.final_output

                grader_scores = await get_eval_grader_score(eval_id=EVAL_ID, summary=summary, section=section)
                average_score = calculate_grader_score(grader_scores)
                total_score = calculate_total_grader_score(grader_scores)
                lenient_passed = is_lenient_pass(grader_scores, average_score)
                print(
                    f"	Scores — avg={average_score:.3f}, total={total_score:.3f}, lenient_passed={lenient_passed}"
                )

                record_aggregate_prompt_score(
                    prompt_version=summarization_prompt.current().version,
                    prompt_text=summarization_prompt.current().prompt,
                    model_name=summarization_prompt.current().model,
                    average_score=average_score,
                    total_score=total_score,
                )

                update_best_candidate(
                    average_score=average_score,
                    prompt_text=summarization_prompt.current().prompt,
                    model_name=summarization_prompt.current().model,
                    summary_text=summary,
                    metadata={
                        "section": section_number,
                        "average_score": average_score,
                        "grader_results": grader_scores,
                        "prompt_version": summarization_prompt.current().version,
                    },
                    lenient_passed=lenient_passed,
                    prompt_version=summarization_prompt.current().version,
                )

                if lenient_passed:
                    optimization_success = True
                    print(f"	Passed with prompt v{summarization_prompt.current().version}")
                    break

                print("	Failed eval. Improving prompt...")
                eval_feedback = collect_grader_feedback(grader_scores)

                metaprompt_result = await Runner.run(
                    metaprompt_agent,
                    input=METAPROMPT_TEMPLATE.format(
                        original_prompt=summarization_prompt.current().prompt,
                        section=section,
                        summary=summary,
                        reasoning=eval_feedback,
                    ),
                )
                improved_prompt = metaprompt_result.final_output
                summarization_prompt.update(
                    new_prompt=improved_prompt,
                    metadata={"section": section, "summary": summary},
                )
                summarization_agent = make_summarization_agent(summarization_prompt.current())

                print(f"	Prompt improved → v{summarization_prompt.current().version}")

            if not optimization_success:
                print(
                    "	All attempts failed; keeping latest prompt version "
                    f"v{summarization_prompt.current().version} for the next section."
                )

    summarization_agent = apply_best_candidate_if_needed()

    print("" + "-" * 80)
    print("Completed optimization loop.")
    print(f"Final prompt version: v{summarization_prompt.current().version}")
    if best_candidate["score"] > float("-inf"):
        print(
            f"Best lenient prompt: v{best_candidate.get('version')} (avg={best_candidate['score']:.3f})"
        )

    aggregate_best = select_best_aggregate_prompt()
    if aggregate_best:
        per_section = (
            aggregate_best.get("total_average", 0.0) / aggregate_best.get("count", 1)
            if aggregate_best.get("count")
            else 0.0
        )
        print(
            f"Aggregate best prompt: v{aggregate_best.get('version')} "
            f"(total={aggregate_best.get('total_score', 0.0):.3f}, avg/section={per_section:.3f}, model={aggregate_best.get('model', 'unknown')})"
        )

    print(f"Final prompt:{summarization_prompt.current().prompt}")
    return summarization_agent

summarization_agent = await self_evolving_loop(summarization_agent)


**最終プロンプトの選択方法**

- すべての評価では、採点者の平均スコア、採点者全体の合計スコア、および試行が寛大な基準を通過したかどうかがログに記録されます。
- `best_candidate`は最新の寛大な合格を追跡しますが（透明性のため）、最終選択では集計された合計値を使用して、全体的に最高性能のプロンプトを確実に保持します。
- ループが終了すると、`apply_best_candidate_if_needed`は最高の累積採点者スコアを持つプロンプトを復元します（同点の場合は最新バージョンを優先）。これにより、表示されるプロンプトが観測された中で最も強力なパフォーマーであることが保証されます。

以下は上記のコードの（要約された）出力例です。

出力を検査すると、自己進化プロンプトが機能したことがわかります。考慮すべきいくつかの要点があります：
1. 最適化は常に成功するとは限らないため、プロンプトのバージョンをロールバックできることが重要です
2. 評価者からの情報の忠実度は、品質の高い最適化を確保するために極めて重要です

### エージェントログとトレーシング

ダッシュボードのログ下で最適化ワークフローの実行を確認できます：

<img src="../../../images/agent_log_traces.png" alt="Agent log traces" style="max-width:50%">
<br><em>図10 - ダッシュボードで最適化ワークフローの実行を示すエージェントログトレース</em>

そして、異なるエージェント呼び出しの詳細を掘り下げることができます：

<img src="../../../images/agent_trace_details.png" alt="Agent trace details" style="max-width:50%">
<br><em>図11 - 個別のエージェント呼び出しと実行フローを示す詳細なエージェントトレース</em>

### 継続的監視

評価ループが完了したら、システムは新しい受信データを継続的に監視し、ブラインドデータセットでモデルのパフォーマンスを定期的に再評価する必要があります。これにより、データ分布が変化してもモデルが正確で準拠した状態を維持できます。

継続的監視を有効にするには、cronジョブまたは軽量なスケジューラーループを統合して、データソースの更新（例：新しいPDFアップロードやデータベースエントリ）を定期的にチェックできます。新しいデータが検出されると、システムは前述の評価と最適化ループを自動的にトリガーします。

例（疑似コード）：

In [ ]:
# this cell is pseudo-code and not meant to be run as-is

import time

def continuous_monitoring(interval_hours=24):
    """Periodically check for new data and trigger the evaluation loop."""
    while True:
        print("Checking for new data...")
        if new_data_detected():
            print("New data found — running evaluation and optimization loop.")
            self_evolving_loop()
        else:
            print("No new data. Sleeping until next cycle.")
        time.sleep(interval_hours * 3600)

continuous_monitoring(interval_hours=24)

このアプローチにより、モデルは継続的に学習し適応することができ、新しいデータを処理するにつれて時間の経過とともに改善されます。これは、高品質で実世界での性能を維持するための重要な要件です。

## 4. さらなる発展

### a. モデル評価

これで、**evals**を使ってプロンプトを改善し、評価が定義された閾値を超えた場合に新しいプロンプトを受け入れる、完全に自動化されたループが完成しました。

本番環境では、新しいユーザーリクエストが入ってくる際に、同様のフレームワークを使用してエージェントのパフォーマンスを監視することができます。
上記で述べたように、これは簡略化された例であり、実際のシナリオでは、新しいプロンプトを承認するために追加のガードレールとhuman-in-the-loopアプローチが必要になります。

この概念をさらに発展させると、evalsを使用してモデルバージョン、冗長性、推論などの異なるモデルパラメータ候補をテストすることもできます。考慮可能なパラメータの完全なセットを確認するには、[Agents SDKのModelSettingsクラス](https://openai.github.io/openai-agents-python/ref/model_settings/#agents.model_settings.ModelSettings)をチェックしてください。

`compare_model_candidates`関数は以下の方法の例です：
1. プロンプトを最適化する
2. 最適化されたプロンプトを使用して、2つ以上の異なるモデルから候補出力を生成する
3. evalsを使用して候補出力を評価し、最適な候補を選択する

この関数は最小限のリファクタリングで`self_evolving_loop`関数に組み込むことができます。

> **注意：** モデルバージョンの本番テストは、同じファミリーバージョン内のバージョン（例：gpt-5、gpt-5-mini、gpt-5-nano）に限定すべきです。ファミリー間のバージョン選択は本番デプロイメント前に実施することを推奨します。

そして、モデル比較コードを含む最終的な `self_evolving_loop`：

In [ ]:
from agents import Agent, Runner

async def eval_agent_candidate(agent: Agent, section: str, prompt_text: str, model_name: str):
    summary_result = await Runner.run(agent, section)
    summary = summary_result.final_output

    scores = await get_eval_grader_score(
        eval_id=EVAL_ID, summary=summary, section=section
    )
    average = calculate_grader_score(scores)
    lenient_passed = is_lenient_pass(scores, average)
    passed = all(entry.get("passed") is True for entry in scores)

    update_best_candidate(
        average_score=average,
        prompt_text=prompt_text,
        model_name=model_name,
        summary_text=summary,
        metadata={
            "section": section,
            "average_score": average,
            "grader_results": scores,
        },
        lenient_passed=lenient_passed,
    )

    return {"summary": summary, "scores": scores, "average": average, "passed": passed}

async def compare_model_candidates(
    summarization_prompt,
    eval_feedback: str,
    section: str,
    summary: str,
    model_candidates=None,
):
    """Improve the prompt, evaluate it across candidate models, and adopt the top performer."""
    if model_candidates is None:
        model_candidates = ["gpt-5", "gpt-5-mini"]

    metaprompt_result = await Runner.run(
        metaprompt_agent,
        input=METAPROMPT_TEMPLATE.format(
            original_prompt=summarization_prompt.current().prompt,
            section=section,
            summary=summary,
            reasoning=eval_feedback,
        ),
    )
    improved_prompt = metaprompt_result.final_output

    async def evaluate_model(model_name: str):
        candidate_agent = Agent(
            name=f"SummarizationAgent:{model_name}",
            instructions=improved_prompt,
            model=model_name,
        )
        result = await eval_agent_candidate(candidate_agent, section, improved_prompt, model_name)
        return model_name, candidate_agent, result

    best = {
        "average": float("-inf"),
        "passed": False,
        "agent": None,
        "model": None,
        "summary": None,
    }

    tasks = [asyncio.create_task(evaluate_model(model_name)) for model_name in model_candidates]
    for task in asyncio.as_completed(tasks):
        model_name, candidate_agent, result = await task
        print(
            f"Candidate average — {model_name}: {result['average']:.4f} "
            f"(passed={result.get('passed', False)})"
        )
        if result["average"] > best["average"]:
            best.update(
                {
                    "average": result["average"],
                    "model": model_name,
                    "summary": result.get("summary"),
                    "agent": candidate_agent,
                    "passed": result.get("passed", False),
                }
            )

    for task in tasks:
        if not task.done():
            task.cancel()

    if best["passed"] and best["model"]:
        summarization_prompt.update(
            new_prompt=improved_prompt,
            model=best["model"],
            metadata={"section": section, "summary": best["summary"]},
        )
        print(f"Updated summarization_prompt with passing model: {best['model']}")
        return make_summarization_agent(summarization_prompt.current())

    print(
        f"No passing models. Best candidate (model={best['model']}, "
        f"avg={best['average']:.4f}) did not pass. Prompt not updated."
    )
    return None

async def self_evolving_loop_with_model_comparison(summarization_agent: Agent) -> Agent:
    print(
        f"Starting self-evolving loop | Initial prompt v{summarization_prompt.current().version}"
    )
    print(f"Prompt: {summarization_prompt.current().prompt}")
    print(f"Model: {summarization_prompt.current().model}")
    print("-" * 80)

    reset_best_trackers()
    df = pd.read_csv("data/dataset.csv")

    with trace("Self-evolving Optimization Workflow: model comparison"):
        for _, row in df.head(5).iterrows():
            content = row.get("content")
            if pd.isna(content) or (isinstance(content, str) and not content.strip()):
                continue

            section_number = str(row["section_number"])
            section = str(content)
            current_version = summarization_prompt.current().version

            print(f"[Section {section_number}] Using prompt v{current_version}")

            summary_passed = False

            for attempt in range(1, MAX_OPTIMIZATION_RETRIES + 1):
                print(f"\tAttempt {attempt}: evaluating summary...")

                summary_result = await Runner.run(summarization_agent, section)
                summary = summary_result.final_output

                grader_scores = await get_eval_grader_score(
                    eval_id=EVAL_ID, summary=summary, section=section
                )
                average_score = calculate_grader_score(grader_scores)
                total_score = calculate_total_grader_score(grader_scores)
                lenient_passed = is_lenient_pass(grader_scores, average_score)
                print(
                    f"\tScores — avg={average_score:.3f}, total={total_score:.3f}, lenient_passed={lenient_passed}"
                )

                record_aggregate_prompt_score(
                    prompt_version=summarization_prompt.current().version,
                    prompt_text=summarization_prompt.current().prompt,
                    model_name=summarization_prompt.current().model,
                    average_score=average_score,
                    total_score=total_score,
                )

                update_best_candidate(
                    average_score=average_score,
                    total_score=total_score,
                    prompt_text=summarization_prompt.current().prompt,
                    model_name=summarization_prompt.current().model,
                    summary_text=summary,
                    metadata={
                        "section": section_number,
                        "average_score": average_score,
                        "grader_results": grader_scores,
                        "prompt_version": summarization_prompt.current().version,
                    },
                    lenient_passed=lenient_passed,
                    prompt_version=summarization_prompt.current().version,
                )

                if lenient_passed:
                    summary_passed = True
                    print(
                        f"\tPassed with prompt v{summarization_prompt.current().version} (model={summarization_prompt.current().model})"
                    )
                    break

                print("\tFailed eval. Improving prompt...")
                eval_feedback = collect_grader_feedback(grader_scores)

                new_agent = await compare_model_candidates(
                    summarization_prompt=summarization_prompt,
                    eval_feedback=eval_feedback,
                    section=section,
                    summary=summary,
                    # model_candidates could be given as an argument if you want to expand options.
                )

                if new_agent is None:
                    print(
                        "\tNo passing model found. Optimization failed for this section."
                    )
                    summary_passed = False
                else:
                    summarization_agent = new_agent
                    summary_passed = True
                    print(
                        f"\tPrompt improved → v{summarization_prompt.current().version} "
                        f"(model={summarization_prompt.current().model})"
                    )
                    break

            if not summary_passed:
                print(
                    "\tAll attempts failed; keeping latest prompt version "
                    f"v{summarization_prompt.current().version} (model={summarization_prompt.current().model}) for the next section."
                )

    summarization_agent = apply_best_candidate_if_needed()

    print("" + "-" * 80)
    print("Completed optimization loop.")
    print(f"Final prompt version: v{summarization_prompt.current().version}")
    print(f"Final model: {summarization_prompt.current().model}")
    aggregate_best = select_best_aggregate_prompt()
    if best_candidate["score"] > float("-inf"):
        print(
            f"Best lenient prompt: v{best_candidate.get('version')} (avg={best_candidate['score']:.3f}, model={best_candidate.get('model', 'unknown')})"
        )
    if aggregate_best:
        per_section = (
            aggregate_best.get("total_average", 0.0) / aggregate_best.get("count", 1)
            if aggregate_best.get("count")
            else 0.0
        )
        print(
            f"Aggregate best prompt: v{aggregate_best.get('version')} "
            f"(total={aggregate_best.get('total_score', 0.0):.3f}, avg/section={per_section:.3f}, model={aggregate_best.get('model', 'unknown')})"
        )
    print(f"Final prompt: {summarization_prompt.current().prompt}")
    print(f"Final model: {summarization_prompt.current().model}")
    return summarization_agent

summarization_agent = await self_evolving_loop_with_model_comparison(summarization_agent)


ここでは、モデルバージョンのスコアに関する追加情報を含む、非常に似た出力を確認できます：

### b. Genetic-Pareto（GEPA）によるプロンプト最適化

自己進化ループが機能し、Evalsを使用してプロンプトを自律的に改善できることを実証しました。しかし、システムプロンプトを改善するために、比較的単純で静的なメタプロンプトに依存していました。このセクションでは、Genetic-Pareto（GEPA）[[1]](##Citations)を使用したより動的で反射的な手法を探求します。GEPAは、エージェントの軌跡をサンプリングし、自然言語でそれらを反映し、プロンプトの修正を提案し、反復的なフィードバックループを通じてシステムを進化させるフレームワークです。

[こちら](https://doi.org/10.48550/arXiv.2507.19457)で入手可能な論文で説明されているGEPA手法は、継続的で自己改善するプロンプト最適化のための魅力的な設計図を提供します。以下のコードは、[こちら](https://github.com/gepa-ai/gepa)で入手可能なGEPA Githubリポジトリを大いに参考にしています。

In [ ]:
import pandas as pd
import gepa
from gepa import EvaluationBatch

# Extract sections from dataset
def read_csv_content(file_path: str) -> list[dict]:
    """Read csv and return section to summarize."""
    df = pd.read_csv(file_path)
    return [{'content': content} for content in df['content'].tolist()]

# Split dataset into training and validation sets
trainset = read_csv_content("data/dataset.csv")
val_cut = max(1, int(0.1 * len(trainset)))
valset = trainset[:val_cut] if len(trainset) > 1 else trainset

私たちは小さなアダプターを追加することで、グレーダーとヘルパー関数を再利用し、セットアップがGEPAで動作するようにします。GEPAの`GEPAAdapter`により、評価フレームワークに簡単に組み込むことができます。3つのフックを定義しました：

- `evaluate`: 前のセクションで定義されたグレーダー（chemical_name_grader、word_length_deviation_grader、cosine_similarity、llm_as_judge）を使用して要約を実行し、評価を行います。
- `get_components_to_update`: GEPAが進化させるべきテキストフィールド（ここではsystem_prompt）を取得します。
- `make_reflective_dataset`: リフレクション用に入力、出力、フィードバックをパッケージ化します。

In [ ]:
class EvalsBackedSummarizationAdapter:
    """
    Minimal adapter for GEPA:
      - evaluate(...) -> EvaluationBatch (scores + outputs + feedback-rich trajectories)
      - get_components_to_update(...) returns the prompt to update
      - make_reflective_dataset(...) packages examples for reflection
    """
    propose_new_texts = None  # use GEPA's default reflection flow

    def __init__(self, client, eval_id: str, gen_model: str = "gpt-5", user_prefix: str | None = None):
        self.client = client
        self.eval_id = eval_id
        self.gen_model = gen_model
        self.user_prefix = user_prefix or "Summarize:\n\n"

    # Same summarization agent as in the previous section
    def _summarize(self, system_prompt: str, section: str) -> str:
        resp = self.client.chat.completions.create(
            model=self.gen_model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"{self.user_prefix}{section}"},
            ],
        )
        return resp.choices[0].message.content.strip()

    # Required by GEPA: run eval minibatch
    def evaluate(self, inputs: list[dict], candidate: dict, capture_traces: bool = True) -> EvaluationBatch:
        system_prompt = candidate["system_prompt"]

        scores: list[float] = []
        outputs: list[str] = []
        trajectories: list[dict] = []

        for item in inputs:
            section = item["content"]

            # 1) Generate with the candidate prompt
            summary = self._summarize(system_prompt, section)
            outputs.append(summary)

            # 2) Grade using previous evals pipeline
            run = run_eval(eval_id=self.eval_id, section=section, summary=summary)
            out_items = poll_eval_run(eval_id=self.eval_id, run_id=run.id)
            grader_scores = parse_eval_run_output(out_items)

            # 3) Score + actionable feedback
            scalar = calculate_grader_score(grader_scores)
            feedback = collect_grader_feedback(grader_scores) or "All graders passed; keep precision and coverage."

            scores.append(float(scalar))
            trajectories.append(
                {
                    "inputs": {"section": section},
                    "generated_output": summary,
                    "metrics": {
                        "combined": float(scalar),
                        "by_grader": grader_scores,  # keeping for analysis if needed
                    },
                    "feedback": feedback,
                }
            )

        return EvaluationBatch(scores=scores, outputs=outputs, trajectories=trajectories)

    # Required by GEPA: text field to evolve
    def get_components_to_update(self, candidate: dict) -> list[str]:
        return ["system_prompt"]

    # Required by GEPA: build the reflective dataset the reflection LM will read
    def make_reflective_dataset(self, candidate: dict, eval_batch: EvaluationBatch, components_to_update: list[str]) -> dict:
        examples = []
        for traj in (eval_batch.trajectories or []):
            examples.append(
                {
                    "Inputs": {"section": traj["inputs"]["section"]},
                    "Generated Outputs": traj["generated_output"],
                    "Feedback": traj["feedback"],
                }
            )
        return {"system_prompt": examples}

アダプターの準備ができたので、比較のために以前の自己進化ループと同じ開始プロンプト（`"You are a summarization assistant. Given a section of text, produce a summary."`）とモデル（ここでは`gpt-5`）を使用してGEPAを実行できます。アダプターインスタンス、シード候補、および訓練/検証セットを`gepa.optimize(...)`に提供します。最適化中、GEPAは候補をスコア付けするためにアダプターを繰り返し呼び出し、フィードバックを反映し、最終的に最良の進化したプロンプトを生成します。

_注意：GEPAの完了には約10-15分かかる場合があります。_

In [ ]:
seed_candidate = {"system_prompt": "You are a summarization assistant. Given a section of text, produce a summary."}

adapter = EvalsBackedSummarizationAdapter(
    client=client,
    eval_id=EVAL_ID,
    gen_model=summarization_prompt.current().model,  
)

# Keeping max_metric_calls small for the cookbook. 
# In practice, use a larger value to allow more optimization iterations.
result = gepa.optimize(
    seed_candidate=seed_candidate,
    trainset=trainset,
    valset=valset,
    adapter=adapter,
    reflection_lm="gpt-5",
    max_metric_calls=10,
    track_best_outputs=True,
    display_progress_bar=True
)

best_prompt = result.best_candidate["system_prompt"]
print("\n=== Best evolved instruction ===\n")
print(best_prompt)

以下は上記のコードの（簡略化された）出力例です：

このクックブックでは、プロンプト最適化に対する3つの異なるアプローチを探求しました：

- **OpenAI Platform Optimizer:** 手動で入力された人間のフィードバック（サムズアップ/ダウンと文章コメント）を含むデータセットで_Optimize_ボタンを使用し、最小限の設定で迅速に強力なプロンプトを生成しました。この手法は高速な反復処理に優れていますが、本番環境に必要な自動化は提供されません。

- **静的メタプロンプトを使用した最適化:** 4つの異なる評価器を組み込んだループにより、手動介入なしで自動探索と反復的な自己改善を可能にしました。しかし、その探索空間は単一の静的メタプロンプトによって制限され、評価はセクションごとに実行されました。その結果、このアプローチは、より広範な汎化を達成するのではなく、即座の評価器フィードバックに過学習するリスクがありました。

- **GEPA最適化:** より構造化された検索プロセスを提供し、反省的な更新は定量的スコアと文章フィードバックの両方によって情報を得て、候補は一つのデータセットで訓練され、別のデータセットで検証されました。この手法は、より堅牢で汎化されたプロンプトを生成し、そのパフォーマンスのより明確な実証的証拠を提供しました。

_注：各手法によって生成されたプロンプトの例は付録で確認できます。_

使用ケースに応じて、速度（OpenAI optimizer）、軽量な自動化（静的メタプロンプト）、または体系的な汎化（GEPA）を優先することができます。実際には、高速な反復処理から始めて反省的最適化に進むことでこれらの手法を組み合わせることで、機敏性とパフォーマンスの両方を実現できます。

Happy coding!

## 貢献者

このクックブックは[Bain](www.bain.com)と[OpenAI](openai.com)の共同コラボレーションに基づいています。

[Calvin Maguranis](https://www.linkedin.com/in/calvin-maguranis-b9956045/)  
[Fanny Perraudeau](https://www.linkedin.com/in/fanny-sabran-perraudeau-494b7573/)   
[Giorgio Saladino](https://www.linkedin.com/in/giorgio-saladino-202/)   
[Shikhar Kwatra](https://www.linkedin.com/in/shikharkwatra/)    
[Valentina Frenkel](https://www.linkedin.com/in/valentina-frenkel/)

## 引用文献

[1] _GEPA: Reflective Prompt Evolution Can Outperform Reinforcement Learning_ by Lakshya A Agrawal, Shangyin Tan, Dilara Soylu, Noah Ziems, Rishi Khare, Krista Opsahl-Ong, Arnav Singhvi, Herumb Shandilya, Michael J Ryan, Meng Jiang, Christopher Potts, Koushik Sen, Alexandros G. Dimakis, Ion Stoica, Dan Klein, Matei Zaharia, Omar Khattab - https://arxiv.org/abs/2507.19457

## 付録

### 出力プロンプトの例：

- **初期プロンプト:**  
```pgsql 
You are a summarization assistant. Given a section of text, produce a summary.
```

- **OpenAI Platform Optimizer:** 
```pgsql 
You are a summarization assistant.
Task: Summarize the provided text concisely and accurately.
Output requirements:
- Output only the summary. Do not add titles, labels (e.g.,
"Summary:"), prefaces, or commentary.
- Preserve the document's structure. If multiple sections/subsections appear, summarize each one.
- Use a numbered list for sections/subsections (use their numbers/titles when present).
- Under each, use short dash bullets for key points.
- If there is only a single short section, return a brief bullet list or 1-2 concise sentences.
- Split any inline lists into separate bullets.
- Use plain, simple language. Keep bullets tight (ideally one line each). Remove redundancy.
- Include important quantitative details (values, units, conditions) and constraints. Do not invent information.
- Keep formatting simple: plain text, "1." numbering and "-" bullets only. No tables or special markup.
- Retain exact technical terms/notation from the source (e.g., chemical names, isotopic labels).
- If a section is explicitly marked "Not applicable," include that status; otherwise do not add it.
```

- **静的メタプロンプト:** 
```pgsql 
You are a technical summarization assistant for scientific and regulatory documentation. Your task is to generate a concise, comprehensive, and fully detailed summary of any scientific, technical, or regulatory text provided. Strictly adhere to the following instructions:

---

**1. Complete and Exact Information Inclusion**  
- Capture *every* explicit fact, technical value, specification, quantity, measurement, regulatory reference, entity, process, site, and contextual detail verbatim from the source text.
- Do not omit or generalize any explicit information, no matter how minor.

**2. Precise Terminology and Named Entity Retention**  
- Reproduce all names of chemicals, drugs, mixtures, buffer components, devices, companies, institutions, regulatory standards, section numbers, and procedural labels *exactly as stated*.
- Report all quantities, measurements, concentrations, ratios, masses, volumes, compositions, pH values, and units precisely as given.
- Do not paraphrase, rename, substitute, or simplify any term or value.

**3. All Procedural Details and Justifications**  
- Explicitly include all described procedures, technical processes (e.g., terminal sterilization, aseptic processing), operational constraints, process justifications, compliance requirements, and standards references.
- Clearly state all reasons provided for choosing or omitting particular methods or processes.

**4. Regulatory and Compliance References**  
- Accurately cite all regulations, standards (e.g., USP <797>), compliance statements, section numbers, and cross-references as in the original.
- Include all explicit mentions of compliance, applicability, and site location details.

**5. Explicit Statements of Absence, Limitations, and Applicability**  
- Clearly state any declarations of absence, inapplicability ("Not applicable"), or limitations exactly as written in the source.

**6. Structural and Organizational Fidelity**  
- Precisely reflect the original document's section and subsection hierarchy, using clear section labels and indentation.
- Present all enumerations, lists, and tabulated data in structured bullet-point or numbered format, organized in accordance with the source document's arrangement.

**7. No Paraphrasing, Summarizing, or Reinterpretation**  
- Do *not* paraphrase, summarize contextually, reinterpret, or alter the meaning or sequence of any content.
- Remove only literal repetitions or redundant phrasing; otherwise, preserve all explicit statements, technical details, and contextual notes.

---

**Summary Output Objective:**  
Produce a summary that delivers the full technical, factual, and regulatory content and structure of the original text, reformatted by eliminating only redundant language. The summary must enable audit, regulatory review, or peer reference without loss of any explicit information or terminology from the source.

---

*Apply these instructions rigorously to every provided document section to ensure scientific and regulatory accuracy and completeness.*
```

- **GEPA optimizer**: 
```pgsql 
You are a domain-aware summarization assistant for technical pharmaceutical texts. Given a "section" of text, produce a concise, single-paragraph summary that preserves key technical facts and exact nomenclature.

Length and format
- Write 1–3 sentences totaling about 45–70 words (target ~60; never exceed 90).
- Use one paragraph; no bullets, headings, tables, or heavy formatting.

Exact names and notation
- Include every chemical name that appears in the section at least once, using the exact original spelling, capitalization, punctuation, isotopic labels, brackets, hyphens, salts, buffer names, and parenthetical qualifiers. Treat distinct case/format variants as distinct names (e.g., [1-13C]pyruvic acid and [1-13C]Pyruvic acid are separate and each must appear once).
- Examples you must preserve verbatim when present: Hyperpolarized Pyruvate (13C) Injection; non-polarized Pyruvate Injection; Pyruvate (13C) Injection; hyperpolarized [1-13C]pyruvate; Mixture of [1-13C]pyruvic acid and 15 mM AH111501 sodium salt; TRIS/EDTA buffer solution; TRIS; NaOH; Na2EDTA; [1-13C]pyruvic acid; AH111501 sodium salt.
- Also preserve exact study identifiers, batch codes, section numbers, regulatory citations, and instrument parameters as written (e.g., GE-101-001, GE-101-003, USP <797>, 3.2.P.5.2.5, FFF106/140-806, FFF106/142-806, 3T MRI, 5 degree RF pulse, TR=3s, 90 degree pulse, 64 averages, TR=10s, 10 μl Gd/ml solution).

Content prioritization (if space is tight)
1) What the section is about (topic/purpose).
2) All named chemical entities and compositions (list all chemical names at least once; include concentrations/amounts if given).
3) Critical process/handling facts (e.g., aseptic processing vs terminal sterilization; ISO classifications; filtration specs; compounding/filling steps; temperatures/times/volumes; storage/administration limits).
4) Container/packaging specifics (e.g., cryovials, "sterile fluid path").
5) Microbiological/testing/regulatory details (e.g., sterility/pyrogenicity testing timing; USP <797>; state board compliance; site/manufacturer if stated).
6) Overages/single-dose formulas and key quantities.

Numerical fidelity
- Preserve all critical numbers and units exactly (e.g., 1.44 g, 27.7 mg, 15 mM, 18 mL, 1.47 g, two 0.2 μm filters, ISO 7, ISO 5, 38 mL).
- Include testing/analysis parameters when present (e.g., polarization/relaxation time (T1); number of spectra; pulse angles; TR values; MRI location relative to clean room).

Style and compression
- Be neutral and factual; do not infer unstated information.
- Consolidate repeated statements; compress lists with commas/semicolons to save words.
- Mention tables/figures only to convey key data; do not reproduce them.
- If many chemicals are present, ensure each distinct name appears once; group them succinctly.
- Avoid symbols or special formatting not in the source text.

Common domain cues to include when present
- Aseptic processing vs terminal sterilization and the rationale/timing (e.g., "tested for sterility and pyrogenicity subsequent to patient administration").
- Environmental/processing controls (ISO 7/ISO 5; LAF unit; filtration; filling/weight targets per cryovial).
- Site/regulatory context (e.g., USP <797>; California State Board of Pharmacy; University of California, San Francisco Department of Clinical Pharmacy).
- Study/kit equivalence statements (e.g., equivalence to GE-101-001/GE-101-003 formulations).
- QC/measurement methods (e.g., capacitive threshold at Administration syringe nominal 38 mL).

Self-check before finalizing
- Does the paragraph contain every distinct chemical name exactly as written in the section (including case and notation variants)?
- Is the summary 45–70 words (≤90), in a single paragraph?
- Are the most critical process/regulatory/testing details and all key numbers preserved without unnecessary verbosity?`
```